
# Chat



TODO: move chat analysis tools here, separate from brian's resume, generalize so history can be found in discord and both bots can benefit from the additional context.




## Persistance




### store conversation

store llm response?

store chat history so that it can be recalled later and added to the context window, this gives the LLM more context about how the conversation is going.


In [ ]:
const path = require('path')
const fs = require('fs')
const selectModel = importer.import('select llm')
const {askLlamaAboutEmotions} = importer.import('ask llm about emotions')
const {ACTIVE_CONVERSATIONS, PROJECT_PATH, DEFAULT_MODEL} = importer.import('general chit chat')


async function storeResponse(user, session, content, context, otr) {
  let promptModel = await selectModel(DEFAULT_MODEL)

  let now = new Date()
  let convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') 
    + '-' + DEFAULT_MODEL
    + '-' + session + '.json')
  if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
    if(fs.existsSync(convoFile)) {
      ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
    } else {
      ACTIVE_CONVERSATIONS[convoFile] = {}
    }
  }

  let contextContainsImage = false
  if(context.startsWith('data:image/png;base64,')) {
    contextContainsImage = true
  }

  let summary
  if(!otr) {
    summary = await promptModel('Summerize this prompt in one short sentence:\n' 
      + content + '\nOnly respond with the summary, no pleasantries.')
  }
  let keywords = await promptModel('List a few key words that categorize this prompt:\n' 
    + content + '\nOnly respond with a single category, no pleasantries.')
  let emotions = await askLlamaAboutEmotions(content)

  ACTIVE_CONVERSATIONS[convoFile][Date.now()] = {
    user: user,
    content: otr ? void 0 : content,
    context: contextContainsImage ? void 0 : context,
    summary: summary,
    keywords: keywords,
    emotions: emotions,
    otr: otr ? true : false,
  }
  fs.writeFileSync(convoFile, JSON.stringify(ACTIVE_CONVERSATIONS[convoFile], null, 4))

}


module.exports = storeResponse



### general discussion

general chit chat?

everything else falls into this category. context will be populated with recent message conversation history.


In [ ]:
const path = require('path')
const fs = require('fs')


const PROJECT_PATH = path.join(__dirname, '..', 'Resources', 'Projects', 'conversations')
const ACTIVE_CONVERSATIONS = {}
const DEFAULT_MODEL = process.env.DEFAULT_MODEL || 'Default'

// TODO: load some previous summaries and contents

async function messageRecents(promptModel, session) {

  if(!session) {
    return ''
  }

  let now = new Date()
  let convoFile = path.join(PROJECT_PATH, now.getFullYear() + '-' 
    + String(now.getMonth() + 1).padStart(2, '0') 
    + '-' + DEFAULT_MODEL
    + '-' + session + '.json')
  // TODO: reload chat
  if(typeof ACTIVE_CONVERSATIONS[convoFile] == 'undefined') {
    if(fs.existsSync(convoFile)) {
      ACTIVE_CONVERSATIONS[convoFile] = JSON.parse(fs.readFileSync(convoFile))
    } else {
      ACTIVE_CONVERSATIONS[convoFile] = {}
    }
  }

  let messageTimes = Object.keys(ACTIVE_CONVERSATIONS[convoFile]).filter(k => k != 'summaries')
  messageTimes.sort((a, b) => b - a)
  let count = 0
  let messages = 'Current date: ' + (new Date).toISOString() 
    + '\nOur recent conversion:\n'
  for(let i = 0; i < messageTimes.length && count < 25; i++) {
    let message = ACTIVE_CONVERSATIONS[convoFile][messageTimes[i]]
    let useSummary = i > 10 || messages.length > 1024 || (message.content && message.content.length > 1024)
    let useKeywords = !message.content
    messages += (message.user ? 'User: ' : 'AI: ') 
      + (useKeywords ? message.keywords : (useSummary ? message.summary : message.content))
      + '\n\n'
    if(messages.length > 2048) {
      break
    }
    count++
  }

  return messages
}

module.exports = {
  messageRecents,
  PROJECT_PATH,
  ACTIVE_CONVERSATIONS,
  DEFAULT_MODEL
}
